In [ ]:
import os, pandas as pd, numpy as np, json, matplotlib.pyplot as plt
from natsort import natsorted
from tqdm import tqdm 
os.chdir("./src/lib/hand_predictor")
from utils.hand.api import ffmpeg4format, mp_kpts_generator, mp_kpts_preprocessing, hand_pos_inference, model_pred_severity
from utils.hand.api import hand_rotation, hand_parameters
from utils.third_party.measurement import measurements
from utils.hand.keypoints import stft_plot, mergePlot_PeakInteRaw
import seaborn as sns

# Preprocessing - workflow

## 1. Video -> MP4 

In [ ]:
# input
root_path = '/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_videos'
# root_path = '/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_videos'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.avi' in each]

# output
temp_path = '/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_videos_mp4'
# temp_path = '/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_videos_mp4'

for filename in tqdm(filename_list):
    filename_ = filename.split('.')[0]
    try:
        video_path = f'{root_path}/{filename}'
        output_path = f'{temp_path}/{filename_}.mp4'
        ffmpeg4format(video_path=video_path, output_path=output_path)
    except:
        print(filename)
#collect_hand_keypoints_pipe()

## 2. MP4 -> mediapipe kpts

In [ ]:
pos_LR = 'Right'
# input
root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_videos_mp4'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.mp4' in each]

# output
output_root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_mp_kpts'
os.makedirs(output_root_path, exist_ok=True)

for filename in tqdm(filename_list):
    video_path = f'{root_path}/{filename}'
    mp_kpts_generator(video_path=video_path, output_root_path=output_root_path, hand_query=pos_LR, logging=False)


## 3. Mediapipe kpts preprocessing (error frame ratio, supp with previous frame, normalize)

In [ ]:
pos_LR = 'Right'

# input
root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_mp_kpts'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.csv' in each ]

# output
output_root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_mp_kpts_preprocessed'
os.makedirs(output_root_path, exist_ok=True)

df_list = []
for filename in tqdm(filename_list):
    csv_input_path = f'{root_path}/{filename}'
    csv_output_path = f'{output_root_path}/{filename}'
    error_frame_ratio = mp_kpts_preprocessing(csv_input_path, csv_output_path, logging=False)
    
    df_list.append([filename, error_frame_ratio])

df = pd.DataFrame(df_list, columns=['filename', 'error_frame_ratio'])
df.to_csv(f"{output_root_path}/../{pos_LR.lower()}hand_filename_ErrorFrameRatio.csv", index=None)


## 4. Merge train-val dataset (label create)

In [ ]:
hand_LR = 'right' # 'left', 'right'
datasets = ['train', 'val']

df_list = []
for dataset in datasets:
    read_txt = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_{dataset}.txt'
    df = pd.read_table(read_txt, sep=' ', header=None, dtype=str)
    df.columns = ['filename', 'label']
    df_list.append(df)

df = pd.concat(df_list, axis=0)
df.reset_index(inplace=True, drop=True)
df.to_csv(f"/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_merge.csv", index=None)

## 5. Merge csvname, labels, error frame ratio inforamtion

In [ ]:
hand_LR = 'left' # 'left', 'right'
csv_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_merge.csv'
csvEFR_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_filename_ErrorFrameRatio.csv'

df_csv = pd.read_csv(csv_path)
df_csvEFR = pd.read_csv(csvEFR_path)

df = pd.concat([df_csvEFR, df_csv], axis=1)
df.columns = range(len(df.columns))
df = df[[0,3,1]]
df.columns = range(len(df.columns))
df.to_csv(f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_clean.csv',header=None, index=None)

## 6. Relabel with dataset

In [ ]:
hand_LR = 'left' # 'left', 'right'
severity_boundary = 5
csv_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_clean.csv'
df = pd.read_csv(csv_path, header=None)
df[1] = [1 if each >= severity_boundary else 0 for each in df[1].values] ## modify here
df.to_csv(f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{hand_LR}hand_clean_{severity_boundary}.csv', index=None, header=None)

## 7. Rotation the hand (optional)

In [ ]:
pos_LR = 'Right' # Right, Left

# input
root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_mp_kpts_preprocessed'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.csv' in each ]

# output
output_root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_mp_kpts_preprocessed_rotat'
os.makedirs(output_root_path, exist_ok=True)

df_list = []
for filename in tqdm(filename_list):
    csv_input_path = f'{root_path}/{filename}'
    csv_output_path = f'{output_root_path}/{filename}'
    data_input = pd.read_csv(csv_input_path)
    data_output = hand_rotation(
        data_input, 
        'xyz',
        # [0, 90, 0], #left
        [0, -90, 0], #Right
        )
    data_output.to_csv(csv_output_path, index=None)


# Predict and Compare

## Hand position inference and compare to UPDRS 0/1 (normal/abnormal)

### Right hand

In [ ]:
severity_boundary = 1
cfg_R, clean_results_R, clean_reuslts_woNA_R = hand_pos_inference(
    test_data_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_mp_kpts_preprocessed",
    test_map_path=f"/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_clean_{severity_boundary}.csv",
    model_path=f"./utils/saved_models/DrGuo_3d_rotat_val_pick/Right_FG_{severity_boundary}/best.pth",
    args_path=f"./utils/saved_models/DrGuo_3d_rotat_val_pick/Right_FG_{severity_boundary}/args.txt",
    #output_folder="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_clean_1_eval",
    seed=42,
    balance_dataset=True,
    multiple_sampling_num=5,
    random_rotat_3d=True,
    gau_samp=False,
    logging=True,
    device='cpu'
)

In [ ]:
_ = measurements(
    clean_reuslts_woNA_R['label'].values,
    clean_reuslts_woNA_R['predict'].values,
    with_auc=False, printout=cfg_R['logging']
)

### Left hand

In [ ]:
severity_boundary = 1
cfg_L, clean_results_L, clean_reuslts_woNA_L = hand_pos_inference(
    test_data_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_mp_kpts_preprocessed",
    test_map_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_clean_1.csv",
    model_path=f"./utils/saved_models/DrGuo_3d_rotat_val_pick/Left_FG_{severity_boundary}/best.pth",
    args_path=f"./utils/saved_models/DrGuo_3d_rotat_val_pick/Left_FG_{severity_boundary}/args.txt",
    #output_folder="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_clean_1_eval",
    seed=42,
    balance_dataset=True,
    multiple_sampling_num=5,
    random_rotat_3d=True,
    gau_samp=False,
    logging=True,
    device='cpu'
)


In [ ]:
_ = measurements(
    clean_reuslts_woNA_L['label'].values,
    clean_reuslts_woNA_L['predict'].values,
    with_auc=False, printout=cfg_L['logging']
)

## Severity 

### Right

In [ ]:
df_R = model_pred_severity(
    test_data_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_mp_kpts_preprocessed",
    test_map_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/righthand_clean.csv",
    hand="Right",
    random_rotat_3d=True,
    seed=42
)

In [ ]:
df_R.head()

In [ ]:
df_R.to_csv('/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/results_seed42/right_PDMotorDB_pred_3d_valpick_enhanced_feat.csv', index=None)

### Left 

In [ ]:
df_L = model_pred_severity(
    test_data_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_mp_kpts_preprocessed",
    test_map_path="/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/lefthand_clean.csv",
    hand="Left",
    random_rotat_3d=True,
    seed=42
)

In [ ]:
df_L.head()

In [ ]:
df_L.to_csv('/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/results_seed42/left_PDMotorDB_pred_3d_valpick_enhanced_feat.csv', index=None)

# Others: Traditional Hand keypoints (amplitude, frequency)

## Transform and Calculate

In [ ]:
pos_LR = 'Right' # Right, Left

# input
root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsonism/Data/PDMotorDB/{pos_LR.lower()}hand_mp_kpts_preprocessed'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.csv' in each ]

# output
output_root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsonism/Data/PDMotorDB/{pos_LR.lower()}hand_parameters_results'
os.makedirs(output_root_path, exist_ok=True)

for filename in tqdm(filename_list):
    results = {}
    csv_input_path = f'{root_path}/{filename}'
    json_output_path = f'{output_root_path}/{filename.replace(".csv", ".json")}'
    data_input = pd.read_csv(csv_input_path)
    results = hand_parameters(data_input=data_input)
    #print(json_output_path)
    with open(json_output_path, "w") as ff:
        json.dump(results, ff, indent=2)

In [ ]:
pos_LR = 'Left' # Right, Left

# input
root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_parameters_results'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.json' in each ]

# output
output_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{pos_LR.lower()}hand_parameters_results.csv'
df_list = []
for filename in tqdm(filename_list):
    json_input_path = f'{root_path}/{filename}'
    with open(json_input_path, "r") as ff:
        results = json.load(ff)
    df_list.append([
        filename, 
        results['freq-mean'], results['freq-std'], results['freq-median'], 
        results['intensity-mean'], results['intensity-std'], results['intensity-median'],
        results['peaks-mean'], results['peaks-std'], results['peaks-median'], 
        results['inte-freq-mean'], results['inte-freq-std'], results['inte-freq-median']])
df = pd.DataFrame(df_list, 
    columns=["filename", 
    "freq-mean", "freq-std", "freq-median", 
    "intensity-mean", "intensity-std", "intensity-median",
    "peaks-mean", "peaks-std", "peaks-median", 
    "inte-freq-mean", "inte-freq-std", "inte-freq-median"]
)
df.to_csv(output_path, index=None)


## Read example

In [ ]:
pos_LR = "Left"
# input
root_path = f'/home/yuy/Project/Gait_proj/fast_eval_Parkinsonism/Data/PDMotorDB/{pos_LR.lower()}hand_parameters_results'
filename_list = [ each for each in natsorted(os.listdir(root_path)) if '.json' in each ]

json_input_path = f"{root_path}/{filename_list[0]}"
with open(json_input_path, 'r') as ff:
    results = json.load(ff)

d, f = np.array(results["distance-thumb-ratio"]), np.array(results["frequency-interval"])
t, max_freq, max_intensity = np.array(results["stft"]["time"]), np.array(results["stft"]["freq"]), np.array(results["stft"]["intensity"])

## Plot

In [ ]:
stft_plot(d)

In [ ]:
mergePlot_PeakInteRaw(
    t, d, max_freq=max_freq, max_intensity=max_intensity,
    inte_ylim_max=0.5
)

# Others: Plot the 3D keypoint from dataset

In [ ]:
from utils.seed import set_seed
from utils.util import parse_args_keypoint
from utils.hand.dataset import PDHandData
import json, pandas as pd

###
severity_boundary=1
handness = 'right'
###

test_data_path=f"/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{handness}hand_mp_kpts_preprocessed"
test_map_path=f"/home/yuy/Project/Gait_proj/fast_eval_Parkinsons/Data/PDMotorDB/{handness}hand_clean_1.csv"
model_path=f"./utils/saved_models/DrGuo_3d_rotat_val_pick/{handness.capitalize()}_FG_{severity_boundary}/best.pth"
args_path=f"./utils/saved_models/DrGuo_3d_rotat_val_pick/{handness.capitalize()}_FG_{severity_boundary}/args.txt"
seed=42
balance_dataset=True
multiple_sampling_num=5
random_rotat_3d=True
gau_samp=False
logging=True
device='cpu'

set_seed(seed) # set seed

# load previous (training setting)
with open(args_path, 'r') as ff:
    cfg = json.load(ff)

# setting
cfg['test_data_path'] = test_data_path
cfg['test_map_path'] = test_map_path
cfg['model_path'] = model_path
cfg['args_path'] = args_path
cfg['output_folder'] = None
cfg['device'] = device
cfg['seed'] = seed
cfg['balance_dataset'] = balance_dataset
cfg['multiple_sampling_type'] = 'random-crop' # no other choice
cfg['multiple_sampling_num'] = multiple_sampling_num
cfg['random_rotat_3d'] = random_rotat_3d
cfg['gau_samp'] = gau_samp
cfg['logging'] = logging
## it is no need to balance dataset when prediction only
cfg['balance_dataset_method'] = 'None' if (cfg['balance_dataset'] == False) else 'random-crop'


# read and preprocessing data map
test_data_df_raw = pd.read_csv(test_map_path, header=None) # read
test_data_df = test_data_df_raw[~test_data_df_raw[0].str.contains("empty")] # filter out empty
if not cfg['low_confid_accept']: test_data_df = test_data_df[~test_data_df[0].str.contains("lowconfid")] # filter out low confidence data
test_data_df.reset_index(inplace=True, drop=True) # reset index


# enhanced features & keypoint selection
# (Priority: enhanced features > keypoint selection)
if cfg['enhanced_feat']:
    enhanced_type = cfg['category'].split("_")[-1]
    all_channels, channels_num = parse_args_keypoint(cfg['keypoint'])
    all_channels = all_channels + [f"enhanced_feat"]
    channels_num = len(all_channels)
else: 
    enhanced_type = 0
    all_channels, channels_num = parse_args_keypoint(cfg['keypoint'])

cfg['enhanced_type'] = enhanced_type
cfg['channels_num'] = channels_num

# classification class
class_map = {each_class: i for i, each_class in enumerate(sorted(test_data_df[1].unique()))}
class_num = len(class_map.keys())

# Check Acceptability domain: error_frame_ratio
if cfg['logging']: print("\nChecking the error frame ratio...")
origin_files_num = len(test_data_df)
test_data_mask = test_data_df[2]<=cfg['error_frame_thres'] # set: error frame ratio <= threshold
file_remained_ratio = sum(test_data_mask)/origin_files_num
test_data_df = test_data_df[test_data_mask] 
if cfg['logging']: print(f"Files remains: {len(test_data_df)}/{origin_files_num} ({file_remained_ratio:.3f})")

# dataset proccessing
test_dataset = PDHandData(filename_label_df=test_data_df,
                                input_channels=all_channels,
                                data_root=test_data_path,
                                seed=cfg['seed'],
                                mk_balanced_dataset=cfg['balance_dataset'],
                                mk_balanced_type=cfg['balance_dataset_method'],
                                multi_sample_type=cfg['multiple_sampling_type'],
                                multi_sample_num=cfg['multiple_sampling_num'],
                                enhanced_type=cfg['enhanced_type'],
                                random_rotat_3d=cfg['random_rotat_3d'],
                                group_map=class_map,
                                gaussian_sampling=cfg['gau_samp'],
                                crop_len=cfg['crop_len'],
                                return_name=True)

In [ ]:
data_test = test_dataset.__getitem__(1)[0]
print(np.shape(data_test))
data_test = pd.DataFrame(data_test.T, columns=test_dataset.input_channels)

In [ ]:
new_data = hand_rotation(
    data_test, 
    rotat_axis='xyz', 
    rotat_angle=[0,90,0])

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

def data_(new=False, time_frame=0):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlim3d(-1.5, 1.5)
    ax.set_ylim3d(-1.5, 1.5)
    ax.set_zlim3d(-1.5, 1.5)
    data = new_data.copy() if new else data_test.copy() 
    xdata = data.iloc[time_frame].filter(regex="x_").values
    ydata = data.iloc[time_frame].filter(regex="y_").values
    zdata = data.iloc[time_frame].filter(regex="z_").values
    ax.scatter3D(xdata, ydata, zdata,  c=range(len(xdata)))

data_(new=False, time_frame=200)
data_(new=True, time_frame=200)

# Comprehensive evaluation

In [ ]:
hand_LR = "Right" # Left, Right
csv_path = f"/home/yuy/Project/Gait_proj/fast_eval_Parkinsonism/Data/PDMotorDB/Hand_PDMotorDB_pred_20221222_{hand_LR.lower()}.csv"
df = pd.read_csv(csv_path)
df.head()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=[8,5],dpi=300)
colors = ["tab:green", "tab:blue", "tab:orange", "tab:red"]
for label, color_ in zip(range(4), colors):
    # df_plot = df[df['label']==label]
    df_plot = df[df['predict_overall_3d_rotat_val_pick_infer_3d_rotat']==label]
    # df_plot = df[df['YYY']==label]
    ax.scatter(df_plot['freq-mean'], df_plot['intensity-mean'], label=f"UPDRS={label}", color=color_)

ax.legend()
ax.set_ylabel("Mean Intensity after STFT (A.U.)")
ax.set_xlabel("Mean Freqeuncy after STFT (Hz)")
ax.set_xlim([0.5,6])
ax.set_ylim([-0.001,0.27])
#ax.set_ylim([-0.001,0.31])
fig.show()



In [ ]:
df.columns

In [ ]:
def quick_compare(xlabel="peaks-mean"):
    fig, axes = plt.subplots(2,2, figsize=[16,9], dpi=300)
    axes = axes.flatten()
    df_plot = df[(df['error_frame_ratio'] <= 0.5) & (df['label'] != 4)]

    sns.kdeplot(df_plot, x=xlabel, hue='label', fill=True, ax=axes[0], legend=False)
    axes[0].legend([f"UPDRS {3-i}" for i in range(4)])
    axes[0].set_title("Evaulated by orignial dataset")

    sns.ecdfplot(df_plot, x=xlabel, hue='label', ax=axes[1])
    axes[1].legend([f"UPDRS {3-i}" for i in range(4)])
    axes[1].set_title("Evaulated by orignial dataset")

    sns.kdeplot(df_plot, x=xlabel, hue='predict_overall_3d_rotat_val_pick_infer_3d_rotat', fill=True, ax=axes[2])
    axes[2].legend([f"UPDRS {3-i}" for i in range(4)])
    axes[2].set_title("Evaulated by DL model")

    sns.ecdfplot(df_plot, x=xlabel, hue='predict_overall_3d_rotat_val_pick_infer_3d_rotat', ax=axes[3])
    axes[3].legend([f"UPDRS {3-i}" for i in range(4)])
    axes[3].set_title("Evaulated by DL model")

    fig.tight_layout()
    fig.show()
    return None

In [ ]:
quick_compare(xlabel="peaks-mean")

In [ ]:
quick_compare(xlabel="intensity-mean")

In [ ]:
quick_compare(xlabel="freq-mean")

In [ ]:
quick_compare(xlabel="inte-freq-mean")